In [4]:
#variables:
#primer dilutions:
#stkprm = 100 #concentration of the stock primer you are adding
#stkvol = 1 #the volume of stock primer you are adding
#dilprm = 2.5 #this is the concentration in uM that you want your working dilution to be

#pcr reaction
# need to get this from the df##Numprimers = 4 #this is how many primers go in each pcr reaction.
#primerconcentration = 0.1 #this is the concentration you want each primer to be in the pcr reaction
#pcrvol = 25 #this is the total volume of your pcr reaction 
#templatengs = .5 #this is the concentration of template you want in your pcr rxn in ng/uL

#template dilutions tells you what the temps need to be diluted to initially so that you can just add 1 uL of template to the pcr:
#need to fill in stock template values further down the script
#diltemp = (templatengs)*(pcrvol)/1

#total_volume = 25
#Q5 = total_volume - (0.5*(total_volume)) #How much Q5 to add
#DPNI = 1 #How much DPNI to add
#DPwater = 19
#cutsmart = 5

#goldengate param inputs
#ngdesired=100



#first import information from the j5 spreadsheet in order to perform appropriate steps
#import feather
#import pyarrow.feather as ft
import pandas
import numpy as np
import os
from datetime import date
 
#for this to work you need to run the python script on the same day that you make the new directory
#today = date.today()
#starter_date = str(today.strftime('%Y%m%d'))
#if folder was created on diff date:
#starter_date = 'typedatehere'
pwd = str(os.getcwd())

def walk_up_folder(path, depth=1):
    _cur_depth = 1        
    while _cur_depth < depth:
        path = os.path.dirname(path)
        _cur_depth += 1
    return path   

paths = pandas.read_csv('C:/users/public/documents/opentrons/paths.csv')
paths



#Input_values = pandas.read_csv('Input.csv') 
Input_values = pandas.read_csv('C:/users/public/documents/opentrons/Golden_Gate/20211019_GoldenGate/Input.csv') 
Input_values
Date = str(int(Input_values.loc[0].at['Date']))
Date

Q5 = (0.5*Input_values.loc[0].at['pcrvol'])
diltemp = (Input_values.loc[0].at['templatengs'])*(Input_values.loc[0].at['pcrvol'])/1


###############################################################################################################################################
#oligos

#os.chdir(paths.loc[0].at['opentrons_repo']+'/Golden_Gate/'+Date+'_Golden_Gate')
os.chdir('C:/users/public/documents/opentrons/Golden_Gate/20211019_GoldenGate/')
os.getcwd()
oligos = pandas.read_csv('oligo.csv')
oligos


oligos['ID Number'] = oligos['ID Number'].astype(int)
oligos

if len(oligos.columns) < 9:
    oligos['well'] = ''
    oligos['stock primer concentration'] = ''
    oligos['volume of stock primer to add'] = ''
    oligos['concentration of diluted primer'] = ''
    oligos['volume of diluted primer'] = '' #this is a calculated value
    oligos['how much of the diluted primer is left'] = '' #also a calculated value
oligos

#custom 4x6 well plate dictionary. hardcoded specifically for the labware used. 
#this could easily be replace with another well specification dictionary

id2well = {}
id2well['0'] = 'A1'
id2well['1'] = 'A2'
id2well['2'] = 'A3'
id2well['3'] = 'A4'
id2well['4'] = 'A5'
id2well['5'] = 'A6'
id2well['6'] = 'B1'
id2well['7'] = 'B2'
id2well['8'] = 'B3'
id2well['9'] = 'B4'
id2well['10'] = 'B5'
id2well['11'] = 'B6'
id2well['12'] = 'C1'
id2well['13'] = 'C2'
id2well['14'] = 'C3'
id2well['15'] = 'C4'
id2well['16'] = 'C5'
id2well['17'] = 'C6'
id2well['18'] = 'D1'
id2well['19'] = 'D2'
id2well['20'] = 'D3'
id2well['21'] = 'D4'
id2well['22'] = 'D5'
id2well['23'] = 'D6'

for i, row in oligos.iterrows():
    oligos.loc[i,'well'] = id2well[str(i)] #this only works because the index matces the id number. id number is a floating value
    oligos.loc[i,'stock primer concentration'] = Input_values.loc[0].at['stkprm']
    oligos.loc[i,'volume of stock primer to add'] = Input_values.loc[0].at['stkvol']
    oligos.loc[i,'concentration of diluted primer'] = Input_values.loc[0].at['dilprm']

    
for i, row in oligos.iterrows():
    oligos.loc[i,'volume of diluted primer'] = row['stock primer concentration']*row['volume of stock primer to add']/row['concentration of diluted primer']
    

oligos['amount primer to add to frag amplification'] = Input_values.loc[0].at['pcrvol']*Input_values.loc[0].at['primerconc']/oligos['concentration of diluted primer']
oligos

oligos.to_csv('output_'+Date+'_oligo_GoldenGate.csv')

###################################################################################################################################################################################
#assembly

#read in assembly pieces as dataframe .... might not need this info
#os.chdir("C:/Users/jonbr/Documents/GitHub/opentrons/Golden_Gate/Part1_PCR_Mason/")
os.getcwd()
assembly = pandas.read_csv('assembly.csv')
assembly

for i, row in assembly.iterrows():
    assembly.loc[i,'pcr_frag_tube'] = id2well[str(i)]
assembly

assembly.to_csv('output_'+Date+'_assembly_GoldenGate.csv')

##############################################################################################################################################################################################
#digests

#os.chdir("C:/Users/jonbr/Documents/GitHub/opentrons/Golden_Gate/Part1_PCR_Mason/")
os.getcwd()
digests = pandas.read_csv('digests.csv')
digests
#digest_conc=138 #automate digest concentration entry with inputs.py
digest_conc = Input_values.loc[0].at['concdigesttemp']
digests['digest_conc']=digest_conc
digests
startnum = len(oligos['well'])
for i, row in digests.iterrows():
    digests.loc[i,'well'] = id2well[str(startnum+ i)]
digests

next_startnum = startnum+len(digests['well'])
digests

for i, row in digests.iterrows():
    digests.loc[i,'amount of template to add'] = 1


for i, row in digests.iterrows():
   
        digests.loc[i,'concentration of template (ng/uL)'] = diltemp

digests['volume of dilute template prepared'] = digests['digest_conc']*Input_values.loc[0].at['stkvol']/digests['concentration of template (ng/uL)']

digests['water to add']= digests['volume of dilute template prepared']-digests['amount of template to add']
digests

for i, row in digests.iterrows():
    digests.loc[i,'frag_pcr_tube'] = id2well[str(i)]
digests

digests.to_csv('output_'+Date+'_digests_GoldenGate.csv')

##################################################################################################################################
#pcr

#os.chdir("C:/Users/jonbr/Documents/GitHub/opentrons/Golden_Gate/Part1_PCR_Mason/")
pcr = pandas.read_csv('pcr.csv')
pcr.columns = pcr.columns.str.replace("'","")
pcr

pcr[['Reaction ID Number','Forward Oligo ID Number','Reverse Oligo ID Number']] = pcr[['Reaction ID Number','Forward Oligo ID Number','Reverse Oligo ID Number']].astype(int)
pcr


#if bumpback > 0:
#    pcr.index = np.arange(1, len(pcr) + 1)
pcr

#here we create an object with each unique template from the
templates = pcr["Primary Template"]
unique_templates = templates.drop_duplicates(keep = 'first', inplace = False)
unique_templates

df = pandas.DataFrame(unique_templates)

df = df.reset_index()
df = df.drop('index', 1)

df['Template Concentration'] = ''
df

#enter template concentrations here
df['Template Concentration'] = Input_values['template concentrations'] #all you have to do is input the template concentrations in the right order

#df = df[df.line_race != 0]


startnum = len(oligos['well'])

for i, row in df.iterrows():
    df.loc[i,'template_well'] = id2well[str(next_startnum+i)]
    


for i, row in df.iterrows():
    df.loc[i,'amount of template to add'] = 1


for i, row in df.iterrows():
   
        df.loc[i,'concentration of template (ng/uL)'] = diltemp

df['volume of dilute template prepared'] = df['Template Concentration']*Input_values.loc[0].at['stkvol']/df['concentration of template (ng/uL)']

df['water to add']= df['volume of dilute template prepared']-df['amount of template to add']


#df['volume of dilute template prepared'] = df['volume of dilute template prepared'].astype(float)
#df['concentration of template (ng/uL)'] = df['concentration of template (ng/uL)'].astype(float)
#df['amount of template to add'] = df['amount of template to add'].astype(float)

#df['water to add'] = df['water to add'].astype(int)
#df['concentration of template (ng/uL)'] = df['concentration of template (ng/uL)'].astype(int)
#df['volume of dilute template prepared'] = df['volume of dilute template prepared'].astype(int)
#df['amount of template to add'] = df['amount of template to add'].astype(int)

df['water to add'].astype(np.float32)
df['amount of template to add'].astype(np.float32)
df['concentration of template (ng/uL)'].astype(np.float32)
df['volume of dilute template prepared'].astype(np.float32)

#df['amount of template to add'] = pd.Series.astype(df['amount of template to add'], dtype=float)
#df['concentration of template (ng/uL)'] = pd.Series.astype(df['concentration of template (ng/uL)'], dtype=float)
#df['volume of dilute template prepared'] = pd.Series.astype(df['volume of dilute template prepared'], dtype=float,)
df

#if bumpback > 0:
#    df.index = np.arange(0+bumpback, len(df) + bumpback)
    
df
#df.to_csv('output_'+Date+'_templates_IVA.csv')

#df.dtypes

#this line of code integrates the template concentrations into the pcr df
pcr_plustemplates = pandas.merge(pcr,df,on='Primary Template')
pcr_plustemplates

if len(pcr_plustemplates.columns) == 17:

    for i, row in pcr_plustemplates.iterrows():
   
        pcr_plustemplates.loc[i,'concentration of template (ng/uL)'] = diltemp

pcr_plustemplates['volume of dilute template prepared'] = pcr_plustemplates['Template Concentration']*Input_values.loc[0].at['stkvol']/pcr_plustemplates['concentration of template (ng/uL)']

pcr_plustemplates

wellinfo = oligos[['ID Number','well']]
wellinfo

wellinfo = wellinfo.rename(columns={'ID Number':'Forward Oligo ID Number'})
pcr_plustemplates = pcr_plustemplates.merge(wellinfo, on= 'Forward Oligo ID Number')
wellinfo = wellinfo.rename(columns={'Forward Oligo ID Number':'Reverse Oligo ID Number','well':'well2'})
pcr_plustemplates = pcr_plustemplates.merge(wellinfo, on= 'Reverse Oligo ID Number')
pcr_plustemplates

#############################################################################################
#remove this!!!
total_volume = 25
#############################################################################################


pcr_plustemplates['total_water_toadd'] = total_volume-Q5-1-1-1
pcr_plustemplates

pcrstart  = len(digests['well'])
for i, row in pcr_plustemplates.iterrows():
    pcr_plustemplates.loc[i,'frag_pcr_tube'] = id2well[str(i+pcrstart)]
pcr_plustemplates

pcr_plustemplates.to_csv('output_'+Date+'_pcr_GoldenGate.csv')

#######################################################################################################################################################################################################################
#combinations

#os.chdir("C:/Users/jonbr/Documents/GitHub/opentrons/Golden_Gate/Part1_PCR_Mason/")
os.getcwd()
combinations = pandas.read_csv('combinations.csv')
combinations


pieces = [columns for columns in combinations if columns.startswith('Assembly Piece ID Number Bin ')]
frame = combinations[pieces]
#frame2 = frame.transpose()
frame

frame += startnum
frame
frame= frame.values.astype(str)
frame = pandas.DataFrame(frame)
frame
result_1 = frame.replace(id2well)
result_1

combinations_plustemplocs = pandas.concat([combinations, result_1], axis=1)
combinations_plustemplocs

pieces = [columns for columns in combinations if columns.startswith('Assembly Piece ID Number Bin ')]
frame = combinations[pieces]
frame2 = frame.transpose()
frame2

#need to remove the row of linearized fragments from the digest when calculating pcr parameters
if assembly.loc[0].at['Reaction Type'] == 'Digest Linearized':
    frame2 = frame2.drop(frame2.index[0])
    frame2 -= 1
frame2

pcr_info = [columns for columns in pcr_plustemplates if columns.startswith('Mean Oligo Tm (3')]
morepcr_info = [columns for columns in pcr_plustemplates if columns.startswith('Delta Oligo Tm (3')]
anotherpcr_info = [columns for columns in pcr_plustemplates if columns.startswith('Length')]
pcr_info = pcr_info + morepcr_info + anotherpcr_info
pcr_info

for column in frame2:
    listoffrags = frame2[column].to_list()
    listoffrags
    
    tablee = pcr_plustemplates[pcr_info]
    tablee = tablee.iloc[listoffrags, :]
    
    if column == 0:
        params0 = tablee.copy()
    if column == 1:
        params1 = tablee.copy()
    if column == 2:
        params2 = tablee.copy()
    if column == 3:
        params3 = tablee.copy()
    if column == 4:
        params4 = tablee.copy()
    if column == 5:
        params5 = tablee.copy()
    if column == 6:
        params6 = tablee.copy()
    if column == 7:
        params7 = tablee.copy()
    if column == 8:
        params8 = tablee.copy()
    if column == 9:
        params9 = tablee.copy()
    if column == 10:
        params10 = tablee.copy()
    if column == 11:
        params11 = tablee.copy()

        
params0

#so looks like this script will be limited for finding pcr conditions for just 2 construct. have to update for more

Lengthparams0 = params0.nlargest(1,'Length')
Lengthparams0['Length'] = (Lengthparams0['Length']/1000)*30
lengthlist = Lengthparams0['Length'].to_list()

Lengthparams1 = params1.nlargest(1,'Length')
Lengthparams1['Length'] = (Lengthparams1['Length']/1000)*30
lengthlist1 = Lengthparams1['Length'].to_list()

Lengthparams2 = params2.nlargest(1,'Length')
Lengthparams2['Length'] = (Lengthparams2['Length']/1000)*30
lengthlist2 = Lengthparams2['Length'].to_list()

Lengthparams3 = params3.nlargest(1,'Length')
Lengthparams3['Length'] = (Lengthparams3['Length']/1000)*30
lengthlist3 = Lengthparams3['Length'].to_list()

#Lengthparams2 = params2.nlargest(1,'Length')
#   Lengthparams2['Length'] = (Lengthparams2['Length']/1000)*60
 #   lengthlist = Lengthparams2['Length'].to_list()
  #  print(lengthlist)

#Lengthparams3 = params0.nlargest(1,'Length')
#Lengthparams0['Length'] = (Lengthparams0['Length']/1000)*60
#lengthlist = Lengthparams0['Length'].to_list()
#print(lengthlist)

#Lengthparams0 = params0.nlargest(1,'Length')
#Lengthparams0['Length'] = (Lengthparams0['Length']/1000)*60
#lengthlist = Lengthparams0['Length'].to_list()
#print(lengthlist)

finallengthlist = lengthlist + lengthlist1 +lengthlist2 + lengthlist3
finallengthlist

combinations['Extension_time_sec'] = finallengthlist
combinations

extens = combinations.nlargest(1,'Extension_time_sec')
extension_final = extens['Extension_time_sec']
extension_final

#solved the problem of not being able to loop through multiple dataframes
params_tables = {'parmx': ['params0','params1','params2','params3']}
params_tables = pandas.DataFrame(data=params_tables)
params_tables

annealing=[]
for i, row in params_tables.iterrows():
    x = params_tables.loc[i].at['parmx']
#locals()[x]
    locals()[x]['Upper_temp'] = locals()[x]['Mean Oligo Tm (3 Only)'] + locals()[x]['Delta Oligo Tm (3Only)']
    locals()[x]['Lower_temp'] = locals()[x]['Mean Oligo Tm (3 Only)'] - locals()[x]['Delta Oligo Tm (3Only)']
    Lowest_high = locals()[x].nsmallest(1,'Upper_temp')
    Lowest_high
    Highest_low = locals()[x].nlargest(1,'Lower_temp')
    Highest_low
    LH = Lowest_high['Upper_temp']
    HL=Highest_low['Lower_temp']
    A = LH-HL
    if A.all() > 0:
        annealing_temp = Lowest_high['Upper_temp']
    if A.all() < 0:
        annealing_temp = (Lowest_high['Upper_temp']+Highest_low['Lower_temp'])/2
    annealing.append(annealing_temp)
#dfff = np.array(annealing)
dfff = pandas.DataFrame(annealing)
dfff

avg_annealing = 68.5925
avg_annealing

Annealing_and_extension = pandas.DataFrame({'Annealing temp': avg_annealing,
                   'extension time (seconds)': extension_final})
Annealing_and_extension = Annealing_and_extension.reset_index()
Annealing_and_extension = Annealing_and_extension.drop(columns = ['index'])
Annealing_and_extension

Annealing_and_extension.to_csv('output_'+Date+'_Annealing_extension.csv')

################################################################################################################################################
#continued combination processing to set up actual Golden Gate assembly dataframes

#os.chdir("C:/Users/jonbr/Documents/Github/opentrons/Golden_Gate/Part2_Assembly_Cam")
#os.getcwd()
#fragments = pandas.read_csv('fragments.csv')
#fragments

#put final goldengate assembly products on row C for good measure

id2wellpcr = {}
id2wellpcr['0'] = 'A1'
id2wellpcr['1'] = 'A2'
id2wellpcr['2'] = 'A3'
id2wellpcr['3'] = 'A4'
id2wellpcr['4'] = 'A5'
id2wellpcr['5'] = 'A6'
id2wellpcr['6'] = 'A7'
id2wellpcr['7'] = 'A8'
id2wellpcr['8'] = 'A9'
id2wellpcr['9'] = 'A10'
id2wellpcr['10'] = 'A11'
id2wellpcr['11'] = 'A12'
id2wellpcr['12'] = 'B1'
id2wellpcr['13'] = 'B2'


combinations


ID_tube = assembly[['Reaction ID Number','pcr_frag_tube']]

ID_tube = ID_tube.rename(columns={'Reaction ID Number':'Assembly Piece ID Number Bin 0'})
combinations = combinations.merge(ID_tube, on= 'Assembly Piece ID Number Bin 0')
ID_tube = ID_tube.rename(columns={'Assembly Piece ID Number Bin 0':'Assembly Piece ID Number Bin 1'})
combinations = combinations.merge(ID_tube, on= 'Assembly Piece ID Number Bin 1')
ID_tube = ID_tube.rename(columns={'Assembly Piece ID Number Bin 1':'Assembly Piece ID Number Bin 2'})
combinations = combinations.merge(ID_tube, on= 'Assembly Piece ID Number Bin 2')

combinations

combs_short = combinations[['pcr_frag_tube_x','pcr_frag_tube_y','pcr_frag_tube']] #,'pcr_frag_tube_y','pcr_frag_tube'
combs_short

combs_short = combs_short.transpose()
combs_short




,0,1,2,3
pcr_frag_tube_x,A1,A1,A1,A1
pcr_frag_tube_y,A2,A2,A2,A2
pcr_frag_tube,A3,A4,A5,A6


In [6]:
gg1 = pandas.DataFrame()
gg2 = pandas.DataFrame()
gg3 = pandas.DataFrame()
gg4 = pandas.DataFrame()

dil_tu = {}
dil_tu['A1'] = 'B1'
dil_tu['A2'] = 'B2'
dil_tu['A3'] = 'B3'
dil_tu['A4'] = 'B4'
dil_tu['A5'] = 'B5'
dil_tu['A6'] = 'B6'
dil_tu['A7'] = 'B7'
dil_tu['A8'] = 'B8'

e = len(combs_short.columns)
next_tc_tube = len(assembly.index)
GG_dfs = {'gg#': ['gg1','gg2','gg3','gg4']}
GG_dfs = pandas.DataFrame(data=GG_dfs)
####################################################################
#Get rid of this
ngdesired = 100
################################################################################
    
for i, row in GG_dfs.iterrows():
    x = GG_dfs.loc[i].at['gg#']
    locals()[x] = combs_short[[i]]
    locals()[x].loc[:,'conc_assumed']= 60

    bps = assembly[['Sequence Length','pcr_frag_tube']]

    bps = bps.rename(columns={'pcr_frag_tube':i})
    locals()[x] = locals()[x].merge(bps, on= i)
    
    backbone_length=locals()[x]["Sequence Length"].max()
    backbone_length
        
    locals()[x] = locals()[x].rename(columns={0:'frag_loc',1:'frag_loc',2:'frag_loc',3:'frag_loc',4:'frag_loc',5:'frag_loc'})
    
       #for i, row in plasmid.iterrows():
       #     plasmid.loc[i,'final tube'] = pcr2final[str(i)]
           
        
    for i, row in locals()[x].iterrows():
        locals()[x].loc[i,'dil_tube'] = dil_tu[locals()[x].loc[i,'frag_loc']]
        
    for i, row in locals()[x].iterrows():
        locals()[x].loc[i,"equimolar ratio"]=locals()[x].loc[i,"Sequence Length"]/backbone_length
        locals()[x].loc[i,"inverse of conc"]=1/locals()[x].loc[i,"conc_assumed"]
        locals()[x].loc[i,"initial required amount"]=ngdesired*locals()[x].loc[i,"equimolar ratio"]*locals()[x].loc[i,"inverse of conc"]
    
        if locals()[x].loc[i,"initial required amount"] > 1: 
            locals()[x].loc[i,"H20 to add to 1uL of fragment"] = 0#np.nan
        if locals()[x].loc[i,"initial required amount"] < 1:
            locals()[x].loc[i,"H20 to add to 1uL of fragment"]=(2/locals()[x].loc[i,"initial required amount"])-1    
    
        locals()[x].loc[i,"new Conc"] = locals()[x].loc[i,"conc_assumed"]/(locals()[x].loc[i,"H20 to add to 1uL of fragment"]+1)
        locals()[x].loc[i,"new required amount"] = ngdesired*locals()[x].loc[i,"equimolar ratio"]*(1/locals()[x].loc[i,"new Conc"])
    
        locals()[x].loc[i,"final amount to add"] = ''
        if locals()[x].loc[i,"initial required amount"] > 1:
            locals()[x].loc[i,"final amount to add"] = locals()[x].loc[i,"initial required amount"]
        else:
            locals()[x].loc[i,"final amount to add"] = locals()[x].loc[i, "new required amount"]
                
#GG_dfs = {'gg#': ['gg1','gg2','gg3','gg4']}
#GG_dfs = pandas.DataFrame(data=GG_dfs)
    
#for i, row in GG_dfs.iterrows():
    #x = GG_dfs.loc[i].at['gg#']
    #for i, row in locals()[x].iterrows():
        locals()[x].loc[i,'location_of_assembly'] = id2wellpcr[str(next_tc_tube)]
    


    next_tc_tube = next_tc_tube + 1


In [7]:
gg4


,frag_loc,conc_assumed,Sequence Length,dil_tube,equimolar ratio,inverse of conc,initial required amount,H20 to add to 1uL of fragment,new Conc,new required amount,final amount to add,location_of_assembly
0,A1,60,4789,B1,1.000000,0.016667,1.666667,0.000000,60.000000,1.666667,1.666667,A10
1,A2,60,793,B2,0.165588,0.016667,0.275980,6.246910,8.279390,2.000000,2.0,A10
2,A6,60,1177,B6,0.245772,0.016667,0.409619,3.882583,12.288578,2.000000,2.0,A10


In [11]:
de = {'cc': [33,0,0], 'dd': [0,31,0], 'ee': [0,0,12]}
de = pandas.DataFrame(data=de)
de

,cc,dd,ee
0,33,0,0
1,0,31,0
2,0,0,12


In [12]:
de = de.sum(axis=1)
dee = de.mean()
dee

25.333333333333332

In [14]:
from tkinter import filedialog
from tkinter import *

def browse_button():
    # Allow user to select a directory and store it in global var
    # called folder_path
    global folder_path
    filename = filedialog.askdirectory()
    folder_path.set(filename)
    print(filename)


root = Tk()
folder_path = StringVar()
lbl1 = Label(master=root,textvariable=folder_path)
lbl1.grid(row=0, column=1)
button2 = Button(text="Browse", command=browse_button)
button2.grid(row=0, column=3)

mainloop()

G:/Shared drives/PlantSynBioLab


In [17]:
import tkinter as tk

root = tk.Tk()
var = tk.StringVar()
user_name = tk.Entry(root,textvariable=var)
user_name.pack()
#var.trace("w",lambda *args: print (var.get()))
var.trace("w", lambda *args: pg.typewrite(var.get(), interval=0.2)

mainloop()

SyntaxError: invalid syntax (<ipython-input-17-e585169166ad>, line 10)

In [26]:
from tkinter import *

ws = Tk()
ws.title("PythonGuides")
ws.geometry('400x300')
ws['bg'] = '#ffbf00'

def printValue():
    pname = player_name.get()
    Label(ws, text=f'{pname}, Registered!', pady=20, bg='#ffbf00').pack()


player_name = Entry(ws)
player_name.pack(pady=30)

Button(
    ws,
    text="Register Player", 
    padx=10, 
    pady=5,
    command=printValue
    ).pack()

ws.mainloop()

dddd = player_name.get()

dddd

TclError: invalid command name ".!entry"

In [19]:
player_name

<tkinter.Entry object .!entry>

In [20]:
print(player_name)

.!entry


In [21]:
print(ws)

.


In [25]:
player_name.get()

TclError: invalid command name ".!entry"